# Post-processing of case study for central plant results

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==3.5.1
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [2]:
import cases as cas
import importlib
import post_process_configurations as con
importlib.reload(con)

<module 'post_process_configurations' from '/home/mwetter/test/thermal-grid-jba-main/PythonResources/RunCases/post_process_configurations.py'>

## Read annual results

This section also clears the old results to free up memory.

In [3]:
# Free up storage, then read new data
import gc
gc.collect()

print("Cleaned up memory.")

Cleaned up memory.


Get list of cases

In [4]:
# Build list of case files, their labels out output file prefix.
# These are used to read in files and create plots.
# This structure allows removing a case from the post-processing in case the simulation did not converge.
import cases as cas
cases = cas.get_cases()

# Store commit in sim_version dictionary
import os
verFil = os.path.join("simulations", "base", "version.txt")
sim_version = dict()
if os.path.exists(verFil):
    with open(verFil, 'r') as fil:
        s = fil.read()
        kv_pairs = [
            tuple(s_str.split('='))
            for s_str in s.split('\n')
        ]
        for ele in kv_pairs:
          if ele[0] != '': # Skips empty line
            sim_version[ele[0]] = ele[1]
          
#print(sim_version)



Read result file. This takes around 3 minutes per result file.

In [5]:

print("Reading result files")
r_base = None
for cas in cases:
    try:
        print(f"Reading {cas['name']}")
        cas['reader'] = con.get_results(cas['name'])
        if cas['name'] == 'base':
            r_base = cas['reader']
    except Exception as e:
        print(f"*** Error reading {cas['name']}: ", e)
        cas['postProcess'] = False
print("Finished reading result files.")
    

Reading result files
Reading base
Reading base_hBor_0_8
Reading base_hBor_1_2
Reading base_dDis_0_8
Reading base_dDis_1_2
Reading base_TCon_10
Reading base_TCon_20
Reading base_TCon_30
Reading base_heaPumSizFac_0_8
Reading base_heaPumSizFac_0_9
Reading base_noEco
Reading heat
Reading cold
Finished reading result files.


In [6]:
AFlo = r_base.max('datDis.AFlo')

print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9:.2f} GWh")
print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9/AFlo*1e6:.2f} kWh/(m a)")

print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/1E6:.2f} million USD per year")
print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/AFlo:.2f} USD/(m2 a)")



Base case: Total energy use: 10.45 GWh
Base case: Total energy use: 93.27 kWh/(m a)
Base case: Total energy cost: 1.54 million USD per year
Base case: Total energy cost: 13.73 USD/(m2 a)


In [7]:
print(f"CPUtime, base {r_base.max('CPUtime')/3600.:.0f} h")

CPUtime, base 10 h


## Write main results to LaTeX file for inclusion in report

In [8]:
import os
importlib.reload(con)

if not os.path.exists("img"):
    os.mkdir("img")
with open(os.path.join("img", "modelicaResults.tex"), "w") as fil:
    # Energy costs
    r_base_dDis_0_8 = None
    for cas in cases:
        if cas['name'] == 'base_dDis_0_8':
            r_base_dDis_0_8= cas['reader']

    ETot_base = r_base.max('ETot.y')
    ETot_dDis_0_8 = r_base_dDis_0_8.max('ETot.y')
    cosETot_base = r_base.max("totEleCos.y")
    dhDis_base = r_base.max('datDis.dhDisAct')
    vDis_base = r_base.max('datDis.vDis_nominal')
    cosETot_dDis_0_8 = r_base_dDis_0_8.max("totEleCos.y")
    dhDis_0_8 = r_base_dDis_0_8.max('datDis.dhDisAct')


    # ALCC is annualized life cycle costs
    # Investment cost difference is length of distribution pipe, time cost difference of 0.5 vs 0.4 m diameter (from Sommer paper) times exchange rate from Euro to $
    invCosDif = 3460 *(1220-1100)*1.15
    # Annualized increase in electricity costs due to smaller pipes.
    delEneCos =  cosETot_dDis_0_8 - cosETot_base
    (ALCC, LCC, I, OM, RC, SR, CRF) = con.calc_finance(invCosDif, 0, 0, 40, 0.01)

    print(f"Annual energy cost difference {delEneCos/1E6:.3f} million $")
    print(f"invCos = {invCosDif/1E6:.3f} million $")
    print(f"ALCC of investment = {ALCC:.0f} $")
    print(f"ALCC of investment and energy {ALCC-delEneCos:.0f} $")

    s = """
\\newcommand{\\modelicaBranch}{""" + f"{sim_version['branch']}" + """\\xspace}
\\newcommand{\\modelicaCommit}{\\href{https://github.com/lbl-srg/thermal-grid-jba/commit/""" + f"{sim_version['commit']}" + """}{""" + f"{sim_version['commit'][0:6]}" + """}\\xspace}

\\newcommand{\\cpuTime}{""" + f"{r_base.max('CPUtime')/3600.:.0f} hours" + """\\xspace}
\\newcommand{\\delEnergyDDisEighty}{$""" + f"{(ETot_dDis_0_8/ETot_base-1)*100:.0f}\%" + """$\\xspace}
\\newcommand{\\delEnergyCosDDisEighty}{$\\$""" + f"{(cosETot_dDis_0_8-cosETot_base)/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}

\\newcommand{\\totEneCosBase}{$\\$""" + f"{cosETot_base/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}
\\newcommand{\\totEneCosDDisEighty}{$\\$""" + f"{cosETot_dDis_0_8/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}

\\newcommand{\\dhBase}{$""" + f"{dhDis_base:.2f} \, \mathrm m ({dhDis_base*3.28084:.2f} \, \mathrm{{ft}})" + """$\\xspace}
\\newcommand{\\dhDDisEighty}{$""" + f"{dhDis_0_8:.2f} \, \mathrm m ({dhDis_0_8*3.28084:.2f} \, \mathrm{{ft}})" + """$\\xspace}
\\newcommand{\\vDisBase}{$""" + f"{vDis_base:.2f} \, \mathrm{{m/s}} ({vDis_base*3.28084:.2f} \, \mathrm {{ft/s}})" + """$\\xspace}

\\newcommand{\\delAnnLifCycEighty}{$\\$""" + f"{(delEneCos-ALCC)/1E6:.2f} \, \mathrm{{million}}" + """$\\xspace}
"""
    fil.write(s)




Annual energy cost difference 0.526 million $
invCos = 0.477 million $
ALCC of investment = 23929 $
ALCC of investment and energy -502147 $


### Write capacities to LaTeX file

In [9]:
# Note that the unyt package used in this function does not allow to reload the module.
# Hence we put the function in its own file
import post_process_write_latex_table as lat
importlib.reload(lat)
lat.write_latex_capacity_table(r_base)

### Energy use


In [10]:
importlib.reload(con)
con.plot_energy(cases)

All electricity use = [10.44570207 10.83572341 10.15579527 13.99599981  9.42348668 10.44988472
 10.50640297 11.30848904 10.48735034 10.48384926 10.62788865 11.0938199
 10.80336435]
Sum of plot = [10.44570252 10.83572419 10.15579577 13.99599964  9.42348701 10.44988492
 10.50640322 11.30848869 10.48735009 10.48384975 10.62788852 11.09382009
 10.80336414]

Heat pumps in ETS   & 4.76 &  42.5 \\
Heat pumps in plant & 3.20 &  28.5 \\
Pumps               & 3.71 &  33.2 \\
Fans                & 0.54 &  4.8 \\
Non-HVAC electricity for buildings & 8.85 &  79.1  \\ \hline
PVs and batteries  & -10.62 &  -94.8 \\
Total & 10.45 &  93.3 \\ \hline


## Loop temperatures

In [11]:
importlib.reload(con)
con.plot_loop_temperatures(cases)


# Demand curves

In [12]:
importlib.reload(con)
con.plotElectricalTimeSeries(r_base)

In [13]:
importlib.reload(con)
con.writeElectricalTimeSeries(r_base)

# Plot plant operation

In [14]:
importlib.reload(con)# List of days to plot
days = [
    {
        "xlim": [31, 32],
        "date": "Feb. 1",
        "name": "Winter"
    },
    {
        "xlim": [160, 161],
        "date": "June 10",
        "name": "Spring"
    },
    {
        "xlim": [213, 214],
        "date": "Aug. 2",
        "name": "Summer"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Controls",
            "y_lim": [-3, 3],
            "factor": 1,
            "offset": 0,
            "vars": [
                {
                    "label": "$P_{st}$",
                    "var": "cenPla.gen.ind.ySt",
                    "linewidth": 1,
                    },
                    {
                    "label": "$P_{pla}$",
                    "var": "cenPla.gen.ind.yPlaOut",
                    }
                ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
                 {
                    "label": "$T_{pla,hea,set}$",
                    "var": "cenPla.gen.ind.TActPlaHeaSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 3
                    },
                 {
                    "label": "$T_{pla,coo,set}$",
                    "var": "cenPla.gen.ind.TActPlaCooSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 1
                    },
                {
                    "label": "$T_{pla,in}$",
                    "var": "TDisWatRet.T",
                    "linewidth": 0.7,
                    "marker": ">",
                    "color": "k"
                    },
                    {
                    "label": "$T_{pla,out}$",
                    "var": "cenPla.gen.senTemGenLea.T",
                    "linewidth": 0.7,
                    "marker": "<",
                    "color": "k"
                    }
            ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
#                    {
#                    "label": "$T_{bor,per,ret}$",
#                    "var": "cenPla.gen.senTemBorPerRet.T",
#                    "marker": "<",
#                    "linestyle": "--",
#                    "color": "g"
#                    },
#                    {
#                    "label": "$T_{bor,cen,sup}$",
#                    "var": "cenPla.gen.senTemBorCenSup.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": ">"
#                    },
#                    {
#                    "label": "$T_{bor,cen,ret}$",
#                    "var": "cenPla.gen.senTemBorCenRet.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": "<"
#                    },
                    {
                    "label": "$T_{hea,pum,sup}$",
                    "var": "cenPla.gen.senTemHeaPumEnt.T",
                    "color": "b",
                    "marker": ">"
                    },
                    {
                    "label": "$T_{hea,pum,ret}$",
                    "var": "cenPla.gen.senTemHeaPumLea.T",
                    "color": "b",
                    "marker": "<"
                    }
                ]
            },
#            {
#            "P_label": "Mass flow\nrate [kg/s]",
#            "P_lim": [0, 500],
#            "factor": 1,
#            "offset": 0,
#            "vars": [
#                    {
#                    "label": "$\dot m_{eco}$",
#                    "var": "cenPla.gen.hex.m2_flow",
#                    },
#                    {
#                    "label": "$\dot m_{bor,per}$",
#                    "var": "cenPla.gen.senTemBorPerRet.port_a.m_flow",
#                    "marker": "o",
#                    },
#                    {
#                    "label": "$\dot m_{bor,cen}$",
#                    "var": "cenPla.gen.senTemBorCenRet.port_a.m_flow",
#                    "marker": "x",
#                    },
#                    {
#                    "label": "$\dot m_{hea,pum}$",
#                    "var": "cenPla.gen.heaPum.m1_flow",
#                    "marker": "v",
#                    }
#                    ]
#            },
            {
            "y_label": "Heat flow\nrate [MW]",
            "y_lim": [-8, 8],
            "factor": 1E-6,
            "offset": 0,
            "vars": [
#                    {
#                    "label": "$\dot Q_{eco}$",
#                    "var": "cenPla.gen.hex.Q2_flow",
#                    },
                    {
                    "label": "$\dot Q_{bor,per}$",
                    "var": "cenPla.borFie.QPer_flow",
                    "marker": "o",
                    },
                    {
                    "label": "$\dot Q_{bor,cen}$",
                    "var": "cenPla.borFie.QCen_flow",
                    "marker": "x",
                    },
                    {
                    "label": "$\dot Q_{hea,pum}$",
                    "var": "cenPla.gen.heaPum.Q1_flow",
                    "marker": "v",
                    }
                    ]
        }
    ]

con.plotPlant(lis, r_base, "plant", days)

In [15]:
importlib.reload(con)
# List of days to plot
days = [
    {
        "xlim": [56, 61],
        "date": "February",
        "name": "February"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Control signal [1]",
            "y_lim": [0, 1],
            "factor": 1,
            "offset": 0,
            "vars": [
                {
                    "label": "$y_{ets,hea,pum,1}$ (ETS)",
                    "var": "bui[1].ets.heaPum.heaPum.ySet",
                },
                {
                    "label": "$y_{ets,hea,pum,2}$ (ETS)",
                    "var": "bui[2].ets.heaPum.heaPum.ySet",
                },
                {
                    "label": "$y_{ets,hea,pum,3}$ (ETS)",
                    "var": "bui[3].ets.heaPum.heaPum.ySet",
                },
                {
                    "label": "$y_{ets,hea,pum,4}$ (ETS)",
                    "var": "bui[4].ets.heaPum.heaPum.ySet",
                },
                {
                    "label": "$y_{ets,hea,pum,5}$ (ETS)",
                    "var": "bui[5].ets.heaPum.heaPum.ySet",
                },
                {
                    "label": "$y_{cen,hea,pum}$ (central plant)",
                    "var": "cenPla.gen.heaPum.ySet",
                    "linewidth": 0.7
                }

                ]
        },
        {
            "y_label": "$\sum P$ [MW]\n(cummulative power)",
            "y_lim": [0, 7],
            "factor": 1E-6,
            "offset": 0,
            "plotSumOfSeries": True,
            "vars": [
                {
                    "label": "$P_{ETS,hea,pum,1}$ (ETS)",
                    "var": "bui[1].ets.heaPum.heaPum.P",
                },
                {
                    "label": "$P_{ETS,hea,pum,2}$ (ETS)",
                    "var": "bui[2].ets.heaPum.heaPum.P",
                },
                {
                    "label": "$P_{ETS,hea,pum,3}$ (ETS)",
                    "var": "bui[3].ets.heaPum.heaPum.P",
                },
                {
                    "label": "$P_{ETS,hea,pum,4}$ (ETS)",
                    "var": "bui[4].ets.heaPum.heaPum.P",
                },
                {
                    "label": "$P_{ETS,hea,pum,5}$ (ETS)",
                    "var": "bui[5].ets.heaPum.heaPum.P",
                },
                {
                    "label": "$P_{cen,hea,pum}$ (central plant)",
                    "var": "cenPla.gen.heaPum.P",
                    "linewidth": 0.7
                }
            ]
        },
        {
            "y_label": "Temperature [$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
                    {
                    "label": "$T_{dis,loo,max}$",
                    "var": "cenPla.TLooMaxMea",
                    "color": "r"
                    },
                    {
                    "label": "$T_{dis,loo,min}$",
                    "var": "cenPla.TLooMinMea",
                    "color": "b"
                    }
                ]
            }
    ]

con.plotPlant(lis, r_base, "peakPower", days, time="days", fontSize=6, nColLegend=1)


# Plot borefield energy

In [16]:
# List of days to plot
days = [
    {
        "xlim": [0, 365],
        "date": "",
        "name": "Annual"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Borefield energy [GWh]",
            "y_lim": [-3, 3],
            "factor": 1/3600/1E9,
            "offset": 0,
            "vars": [
                {
                    "label": "Modelica $E_{bor}$",
                    "var": "EBor.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,per}$",
                    "var": "EBorPer.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,cen}$",
                    "var": "EBorCen.y",
                    "linewidth": 1
                    },
                    {
                    "label": "MILP $E_{bor}$",
                    "var": "borMil.E",
                    "linestyle": "-",
                    "mark_every": 50,
                    "linewidth": 1
                    }
                ]
        }
]

con.plotOneFigure(lis, r_base, "borefieldEnergy", days)

## Sensitivities

### Borefield depth

In [75]:
importlib.reload(con)

dic = {
      "y_label": "electricity\n[GWh/a]",
      "y_lim": [0, 18],
      "factor": 1E-9/3600,
      "unit": "GWh/a",
      "factor2": 1E-3,
      "unit2": "k$/a",
      "offset": 0,
      "x":{
          "var": "cenPla.borFie.hBor",
          "label": "Borefield depth [m]",
          "x_lim": [65, 115],
          "factor": 1,
          "offset": 0
      },
      "vars": [
         {
            "label": "site electricity",
            "var": "ETot.y",
            "linewidth": 1,
            "color": "r",
         },
         {
            "label": "central plant heat pumps",
            "var": "EComPla.y",
            "linewidth": 1,
            "color": "r",
         }             
      ],
      "y2_label": "annualized life cycle\ncost difference [k$\$$/a]",
      "y2_lim": [-30, 60],
      "idxBaseCase": 1, # Zero-based index of base case
      "vars2": [
          {
              "label": "annualized life cycle\ncost difference",
              "energyCost": "totEleCos.y",
              "unitChange": "cenPla.borFie.hBor",
              "costPerUnitChange": 2376*42, # 2376 wells and $42 per meter, from ORNL https://info.ornl.gov/sites/publications/Files/Pub107271.pdf
              "lifeTime": 40,
              "operationAndMaintenance": 0.01
          }
      ]

    }

lis = []
for name in ["base_hBor_0_8", "base", "base_hBor_1_2"]:
   for cas in cases:
      if cas['name'] == name:
            lis.append(cas['reader'])
con.plot_sensitivities(lis, dic, "boreLength")

## District loop sizing


In [76]:
importlib.reload(con)

dic = {
      "y_label": "electricity\n[GWh/a]",
      "y_lim": [0, 18],
      "factor": 1E-9/3600,
      "unit": "GWh/a",
      "factor2": 1E-3,
      "unit2": "k$/a",
      "offset": 0,
      "x":{
          "var": "datDis.dhDisAct",
          "label": "Diameter district loop pipe [m]",
          "x_lim": [0.3, 0.7],
          "factor": 1,
          "offset": 0
      },
      "vars": [
         {
            "label": "site electricity",
            "var": "ETot.y",
            "linewidth": 1,
            "color": "r",
         },
         {
            "label": "district loop pump",
            "var": "EPumDis.y",
            "linewidth": 1,
            "color": "r",
         }             
      ],
      "y2_label": "annualized life cycle\ncost difference [k$\$$/a]",
      "y2_lim": [-800, 1000],
      "idxBaseCase": 1, # Zero-based index of base case
      "vars2": [
          {
              "label": "annualized life cycle\ncost difference",
              "energyCost": "totEleCos.y",
              "unitChange": "datDis.dhDisAct",
              # Investment cost difference per unit change is length of distribution pipe,
              # times cost difference of 0.5 vs 0.4 m diameter (from Sommer paper) times exchange rate from Euro to $
              "costPerUnitChange": 3460 *((1220-1100)/(0.5-0.4))*1.15,
              "lifeTime": 40,
              "operationAndMaintenance": 0.01

          }
      ]
    }

lis = []
for name in ["base_dDis_0_8", "base", "base_dDis_1_2"]:
   for cas in cases:
      if cas['name'] == name:
            lis.append(cas['reader'])
con.plot_sensitivities(lis, dic, "districtLoopDiameter")

## Minimum condenser leaving water temperature

In [96]:
importlib.reload(con)

dic = {
      "y_label": "electricity\n[GWh/a]",
      "y_lim": [0, 18],
      "factor": 1E-9/3600,
      "unit": "GWh/a",
      "factor2": 1E-3,
      "unit2": "k$/a",
      "offset": 0,
      "x":{
          "var": "bui[1].ets.datHeaPum.TConLvgMin",
          "label": "Minimum condenser leaving water temperature [$^\circ \mathrm{C}$]",
          "x_lim": [8, 33],
          "factor": 1,
          "offset": -273.15
      },
      "vars": [
         {
            "label": "site electricity",
            "var": "ETot.y",
            "linewidth": 1,
            "color": "r",
         }             
      ],
      "y2_label": "annualized life cycle\ncost difference [k$\$$/a]",
      "y2_lim": [-800, 1000],
      "idxBaseCase": 1, # Zero-based index of base case
      "vars2": [
          {
              "label": "annualized life cycle\ncost difference",
              "energyCost": "totEleCos.y",
              "unitChange": "bui[1].ets.datHeaPum.TConLvgMin",
              "costPerUnitChange": 0,
              "lifeTime": 20,
              "operationAndMaintenance": 0
              
          }
      ]
    }
lis = []
for name in ["base_TCon_10", "base", "base_TCon_20", "base_TCon_30"]:
   for cas in cases:
      if cas['name'] == name:
            lis.append(cas['reader'])
con.plot_sensitivities(lis, dic, "minimumCondenserLeaving")


x = [ 9.9999939 14.9999939 19.9999939 29.9999939]


## Sensitivity to heat pump sizing

In [95]:
importlib.reload(con)

dic = {
      "y_label": "hours of setpoint violation\n[h/a]",
      "y_lim": [0, 18],
      "factor": 8760,
      "unit": "h/a",
      "factor2": 1E-3,
      "unit2": "k$/a",
      "offset": 0,
      "idxBaseCase": 2, # Zero-based index of base case
      "x":{
          "var": "cenPla.gen.heaPum.QCoo_flow_nominal",
          "label": "Heat pump capacity [MW]",
          "x_lim": [4, 10],
          "factor": -1E-6,
          "offset": 0
      },
      "vars": [
         {
            "label": "set point violation",
            "var": "conVio.y",
            "linewidth": 1,
            "color": "r",
         }             
      ],
      "y2_label": "annualized life cycle\ncost difference [k$\$$/a]",
      "y2_lim": [-600, 200],
      "vars2": [
          {
              "label": "annualized life cycle\ncost difference",
              "energyCost": "totEleCos.y",
              "unitChange": "datDis.QPlaHeaPumCoo_flow_nominal",
              "costPerUnitChange": -1.631,
              "lifeTime": 20,
              "operationAndMaintenance": 0.02
          }
      ]
    }
lis = []
for name in ["base_heaPumSizFac_0_8", "base_heaPumSizFac_0_9", "base"]:
   for cas in cases:
      if cas['name'] == name:
            lis.append(cas['reader'])
con.plot_sensitivities(lis, dic, "centralPlantHeatPump_violation")

# Create second plot with maximum loop temperature
dic["y_label"] = "maximum loop temperature\n[$^\circ \mathrm{C}$]"
dic["y_lim"] = [23, 27]
dic["unit"] = "[$^\circ \mathrm{C}$]"
dic["factor"] = 1
dic["offset"] = -273.15
dic["vars"] = \
       [
         {
            "label": "maximum loop temperature",
            "var": "cenPla.TLooMaxMea",
            "operation": "max",
            "linewidth": 1,
            "color": "r",
         }             
      ]


dic["y2_label"] = "investment\ncost difference [k$\$$/a]"
dic["y2_lim"] = [-6000, 6000]
dic["vars2"] = [
          {
              "label": "investment\ncost difference",
              "energyCost": "cenPla.gen.ind.zer.k",
              "unitChange": "datDis.QPlaHeaPumCoo_flow_nominal",
              "costPerUnitChange": -1.631,
              "lifeTime": 1,
              "operationAndMaintenance": 0
          }
      ]


con.plot_sensitivities(lis, dic, "centralPlantHeatPump_maxLoopTemp")

x = [6.4589795 7.266352  8.0737245]
x = [6.4589795 7.266352  8.0737245]
